# Spell Checker

This task will involve in the creation of a spellchecking system and an evaluation of its performance. 

The training labelled data is in  `holbrook.txt`

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
Here we are not treating these as separate words instead we treat them like a single token.

Here we are using NLTK Library functions to build our spell checker model

## Task 1

The first task is to build a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. 

In [1]:
import nltk

In [2]:
def parser(file):
    lines = open(file).readlines()
    data = []
    # Write your code here

    for line in lines:
        #print(line)
        indexes=[]
        tokens = nltk.word_tokenize(line) # Tokenize each sentence
        #print(tokens)
        original = tokens
        corrected = tokens
        #Split the data by the '|' separator to get the Original and Corrected Sentences
        for token in tokens:
            if("|" in token):
                index_ind = tokens.index(token)
                word_orig = tokens[tokens.index(token)].split('|')[0]
                word_corrected = tokens[tokens.index(token)].split('|')[1].replace("_"," ")
                original = [word_orig if word==tokens[tokens.index(token)] else word for word in original]
                corrected = [word_corrected if word==tokens[tokens.index(token)] else word for word in corrected]
                indexes.append(index_ind)
        data_local = {
            'original':original,
            'corrected':corrected,
            'indexes': indexes
        }
        data.append(data_local)
    return data
    
data = parser("holbrook.txt")
print(parser("holbrook.txt")[2])

assert(parser("holbrook.txt")[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

{'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'indexes': [9]}


The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [3]:
test = data[:100]
train = data[100:]

## **Task 2 : Implement Unigram Model** : 
This task will calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.


In [4]:
from collections import Counter

def unigram(word):
    # Write your code here.
    word = word.lower()
    words_corrected =[]
    for data_ind in train:
        words_corrected.append([element.lower() for element in data_ind['corrected']])
    word_corrected_list = [item for sublist in words_corrected for item in sublist]
    return Counter(word_corrected_list)[word]
    

def prob(word):
    # Write your code here.
    words_corrected =[]
    word = word.lower()
    for data_ind in train:
        words_corrected.append([element.lower() for element in data_ind['corrected']])
    word_corrected_list = [item for sublist in words_corrected for item in sublist]
    return unigram(word)/sum(Counter(word_corrected_list).values())  # Round Upto 6 digits

# Test your code with the following
assert(unigram("me")==87)
print(unigram("me"))
print(prob("me"))


87
0.003989361702127659


## **Task 3** : 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [5]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))


4


Build a function that calculates all words with *minimal* edit distance to the misspelled word. Steps are as follows:

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value


In [6]:
# Function to get the unique from a list preserving the order
def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def get_candidates(token):
    # Write your code here.
    words_corrected =[]
    for data_ind in train:
        words_corrected.append([element for element in data_ind['corrected']])
    word_corrected_list = [item for sublist in words_corrected for item in sublist]
    # Get the unique list of words
    word_corrected_list_unique = unique(word_corrected_list) 
    # Get the distance w.r.t the first token
    dist =edit_distance(token,word_corrected_list_unique[0])
    min_dist_word =[]
    for word in word_corrected_list_unique:
        # Compare the distance with the subsequent tokens and update the list accordingly
        if(edit_distance(token,word) < dist):
            dist = edit_distance(token,word)
            min_dist_word.clear()
            min_dist_word.append(word)
        elif(edit_distance(token,word) == dist):
            min_dist_word.append(word)
    return min_dist_word
        
# Test your code as follows
assert get_candidates("minde") == ['mine', 'mind']
print(get_candidates("minde"))

['mine', 'mind']


## Task 4 :

Create a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 


In [7]:
def correct(sentence):
    # Write your code here
    sentence_corrected = []
    words_corrected =[]
    for data_ind in train:
        words_corrected.append([element for element in data_ind['corrected']])
    word_corrected_list = [item for sublist in words_corrected for item in sublist]
    # Get the list of unique tokens
    word_corrected_list_unique = list(set(word_corrected_list))
    for word in sentence:
        #If the word is present in the unique corpus directly return the word
        if(word in word_corrected_list_unique):
            sentence_corrected.append(word)
        else:
            # Case when get_candidates returns multiple words
            if(len(get_candidates(word)) >1 ):
                #Calculate the probablities of all the words
                prob_list = list(map(prob,get_candidates(word)))
                #Get the word with the maximum probablity
                word_correct = get_candidates(word)[prob_list.index(max(prob_list))]
                sentence_corrected.append(word_correct)
            # Case when get_candidates returns a single word
            else:
                sentence_corrected.append(get_candidates(word)[0])
    return sentence_corrected

assert(correct(["this","whitr","cat"]) == ['this','white','cat']) 

## **Task 5** :Accuracy Calculation: 

#### Applied Methodology 
1. Pass the Incorrect Corpus from the Training Set to the Correct Method
2. Compare the Corrected Output with the Original Corrected Data 
3. Accuracy calculated as = Sum(All the words from the Generated output matches with the Original Corrected Data) / Sum(All words in the Original Corrected Data)

In [8]:
import time

def accuracy(test,correct):
    # Write your code here
    total_count = 0
    corrected_count = 0
    for data_ind in test:
        #print(data_ind['corrected'])
        #print(data_ind['original'])
        index = len(data_ind['original'])
        # Pass each Original sentence through Correct Method to get the corrected sentence
        corrected_list = correct(data_ind['original'])
        #print(corrected_list)
        i = 0
        while(i<index):
            #Compare each word of the Generated Output with the Corrected List
            if(corrected_list[i] == data_ind['corrected'][i]):
                corrected_count += 1
                total_count += 1
            else:
                total_count += 1
            i +=1
    return round(corrected_count/total_count, 5)

In [9]:
# Print the accuracy and the total execution time
xtime = time.time()
print("Accuracy : " + str(accuracy(test,correct)*100) + " percent")
ytime = time.time()
print("Time Elapsed: " + str(ytime-xtime) + " secs")

Accuracy : 81.162 percent
Time Elapsed: 96.22455143928528 secs


## **Task 6 :**

This task is to modify the Algorithm developed to improve the accuracy of the Spell Checker


#### Corrective Approach

To improve the performance of the classifier I have gone through the following options :
1. <b> Add One Smoothing </b> - This adds up one to every word count so that it will not make the probablity as zero, but in our classifier we are using get_candidate method which will always return a value so add one smoothing will not improve the performance.
2. <b> N - Gram Approximation </b> - in place of using unigram this approach will use n-grams i.e. considering n-1 previous tokens to calculate the probablities and passing the n-gram tokens to get_candidates to get the nearest match, but as the corpus is too small using bigrams or trigrams will not improve the performance
3. <b> Parts Of Speech Tagging </b> - This method tags the corresponding Part Of Speech to the words of a sentence. As the classifier is trying to predict word by word its not considering the contextual data. So incorporate the context of words in the sentence we can consider the POS tag of the word in the sentence and ignore the Proper Nouns, Proper Pronouns, Cardinal Numbers while passing the sentence through classifiers. We will consider this as an improvement approach.
4. <b> Stemming </b> - Stemming will return the root of a word. As the classifier is only predicting on the basis of the minimal distance from the words present in the train corpus it may change the word which is spelled correctly and is used in correct context just because the word is not present in the Training Corrected Corpus. So we can ignore those words whose stem is in the dictionary it will prevent some of the changes mentioned previously. <br/><br/>
So as per the options mentioned above I followed the following additional steps - 

    * Pass the sentence through the NLTK Parts Of Speech Tagging 
    * Identify the Proper noun, singular (NNP) or Proper noun, plural (NNPS) or Cardinal number (CD)
    * Ignores the words with the NNP, NNPS, CD while passing the words through the classifier
    * Check the stems of the words in the NLTK Dictionary, if the root is present in the dictionary ignore the words from the classifier


In [10]:
from nltk import pos_tag
from nltk import PorterStemmer
from nltk.corpus import words

In [11]:
def correct_modified(sentence):
    sentence_corrected = []
    words_corrected = []
    for data_ind in train:
        words_corrected.append([element for element in data_ind['corrected']])
    word_corrected_list = [item for sublist in words_corrected for item in sublist]
    word_corrected_list_unique = list(set(word_corrected_list))
    sentence = pos_tag(sentence)
    stemmer = PorterStemmer()
    for word in sentence:
        if(word[0] in word_corrected_list_unique):
            sentence_corrected.append(word[0])
        #elif(word[1] in ('CD','NNP','.', 'PRP','PRP$') ):
        elif(word[1] in ('CD','NNP','.') ):
            sentence_corrected.append(word[0])
        elif(stemmer.stem(word[0].lower()) in words.words()):
            sentence_corrected.append(word[0])
        else:
            candidate_list = get_candidates(word[0])
            if(len(candidate_list) >1 ):
                prob_list = list(map(prob,candidate_list))
                word_correct = candidate_list[prob_list.index(max(prob_list))]
                sentence_corrected.append(word_correct)
            else:
                sentence_corrected.append(candidate_list[0])
    return sentence_corrected

## **Task 7: Test the New Method :**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [12]:
# Print the accuracy and the total execution time
xtime = time.time()
print("Accuracy : " + str(accuracy(test,correct_modified)*100) + " percent")
ytime = time.time()
print("Time Elapsed: " + str(ytime-xtime) + " secs")

Accuracy : 89.877 percent
Time Elapsed: 25.87539529800415 secs


So we have found after using the POS Tagging and Stemming the accuracy of the classifier has improved from 81% to 89% and the total execution time has also decreased from 93 sec to 25 sec